In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,models,transforms
import torch.nn.functional as F
from PIL import Image
from torch.optim import lr_scheduler
import json

### some variables declared for the helper functions below to make and train the model

In [27]:
gpu = True                              #can be True or False if you have a CUDA enabled gpu. I have NVIDIA 1060 so I have declared it as True
arch = "vgg"                            #can be vgg or densenet for the purpose of this propject. I couldnt experiment with resnet or inception model due to the training time required
lr = 0.001                              #learinng rate
hidden_units = 100                      #number of nodes to use in each hideen layers on top of the pretrained model
epochs = 10                             #number of iterations
data_dir = "flowers"                    #PATH of the data folder 
saved_model = "model.pth"               #name of the checkpoint file to be saved as
nThreads = 4                            #number of threads to use for parallel processing
batch_size = 8                          #mini batch size for the gradient updates
use_gpu = torch.cuda.is_available()     #to check the cuda eanbled gpu is available

### Function for Loading the Data

In [29]:
def data(data_dir):
    data_dir = data_dir
    train_dir = data_dir + '/train'
    valid_dir = data_dir + '/valid'
    test_dir = data_dir + '/test'

    train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

    valid_transforms = transforms.Compose([transforms.Resize(256),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

    test_transforms = transforms.Compose([transforms.Resize(256),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

    image_datasets = dict()
    image_datasets['train'] = datasets.ImageFolder(train_dir, transform=train_transforms)
    image_datasets['valid'] = datasets.ImageFolder(valid_dir, transform=valid_transforms)
    image_datasets['test'] = datasets.ImageFolder(test_dir, transform=test_transforms)

    dataloaders = dict()
    dataloaders['train'] = torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True)
    dataloaders['valid'] = torch.utils.data.DataLoader(image_datasets['valid'], batch_size=batch_size)
    dataloaders['test']  = torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size)

    return dataloaders, image_datasets

### Function for the Feedforward and Backpropagation Operation

In [30]:
def train_model( model, criterion, optimizer, scheduler, epochs=25):
    
    dataloaders, image_datasets = data(data_dir)
    if gpu:
        if use_gpu:
            model = model.cuda()
            print ("Using GPU: "+ str(use_gpu))
        else:
            print("Using CPU since GPU is not available/configured")
    print_every = 50
    steps = 0
    for e in range(epochs):
        current_loss = 0
        for images,labels in dataloaders['train']:
            steps += 1
            images,labels = images.to('cuda'),labels.to('cuda')
            optimizer.zero_grad()
            outputs = model.forward(images)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            current_loss += loss.item()
            if steps%print_every == 0:
                model.eval()
                vloss = 0
                accuracy = 0
                for vimages,vlabels in dataloaders['valid']:
                    optimizer.zero_grad()
                    vimages,vlabels = vimages.to('cuda:0'),vlabels.to('cuda:0')
                    model.to('cuda:0')
                    with torch.no_grad():
                        voutputs = model.forward(vimages)
                        vloss = criterion(voutputs,vlabels)
                        ps = torch.exp(voutputs).data
                        eq = (vlabels == ps.max(1)[1])
                        accuracy += eq.type_as(torch.FloatTensor()).mean()
                vloss = vloss/len(dataloaders['valid'])
                accuracy = accuracy/len(dataloaders['valid'])
                print("epoch {}/{} :".format(e+1,epochs),
                        " training loss = {}".format(current_loss/print_every),
                        " validation loss = {}".format(vloss),
                        " accuracy = {:.2f} %".format(accuracy*100))
                current_loss = 0
    return model

### Function Wrapper for Loading Data, Building Model, Training Model and Saving the Model

In [31]:
def train_model_wrapper(arch):
    dataloaders, image_datasets = data(data_dir)
    if arch == 'vgg': 
        model = models.vgg16(pretrained=True)
    elif arch == 'densenet':
        model = models.densenet121(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.classifier[0].in_features
    from collections import OrderedDict
    classifier = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(num_features, 512)),
                              ('relu', nn.ReLU()),
                              ('drpot', nn.Dropout(p=0.5)),
                              ('hidden', nn.Linear(512, hidden_units)),                       
                              ('fc2', nn.Linear(hidden_units, 102)),
                              ('output', nn.LogSoftmax(dim=1)),
                              ]))

    # Reserve for final layer: ('output', nn.LogSoftmax(dim=1))
        
    model.classifier = classifier    
    if gpu:
        if use_gpu:
            model = model.cuda()
            print ("Using GPU: "+ str(use_gpu))
        else:
            print("Using CPU since GPU is not available/configured")
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=lr)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    model = train_model( model, criterion, optimizer, exp_lr_scheduler,epochs=epochs)

    #CHECKPOINT
    model.class_to_idx = dataloaders['train'].dataset.class_to_idx
    model.epochs = epochs
    checkpoint = {'input_size': [3, 224, 224],
                  'batch_size': dataloaders['train'].batch_size,
                  'output_size': 102,
                  'arch': arch,
                  'state_dict': model.state_dict(),
                  'optimizer_dict':optimizer.state_dict(),
                  'class_to_idx': model.class_to_idx,
                  'epoch': model.epochs}
    torch.save(checkpoint, saved_model)

### Label Mapping

In [ ]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

### Training the Model using the Wrapper Function

In [32]:
train_model_wrapper(arch)

Using GPU: True
Using GPU: True
epoch 1/10 :  training loss = 4.911722822189331  validation loss = 0.0481501966714859  accuracy = 19.66 %
epoch 1/10 :  training loss = 3.8260649919509886  validation loss = 0.04654465988278389  accuracy = 24.03 %
epoch 1/10 :  training loss = 3.262538559436798  validation loss = 0.03038039430975914  accuracy = 40.53 %
epoch 1/10 :  training loss = 2.924436345100403  validation loss = 0.033538028597831726  accuracy = 38.96 %
epoch 1/10 :  training loss = 2.5339330720901487  validation loss = 0.03253711014986038  accuracy = 44.66 %
epoch 1/10 :  training loss = 2.2668577361106874  validation loss = 0.024925580248236656  accuracy = 51.58 %
epoch 1/10 :  training loss = 2.1839340460300445  validation loss = 0.024138793349266052  accuracy = 56.67 %
epoch 1/10 :  training loss = 2.083612319231033  validation loss = 0.03181729465723038  accuracy = 57.04 %
epoch 1/10 :  training loss = 1.9842676758766173  validation loss = 0.03853720799088478  accuracy = 58.01 

In [3]:
use_gpu = torch.cuda.is_available

### Function for Processing Image for Testing 

In [4]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    img = Image.open(image)
    img_transform = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485,0.456,0.406],
                                                           [0.229,0.224,0.225])])
    return img_transform(img)

### Loading the Model Parameters Saved after Training

In [5]:
def load_checkpoint(saved_model):
    checkpoint_provided = torch.load(saved_model)
    if checkpoint_provided['arch'] == 'vgg':
        model = models.vgg16()        
    elif checkpoint_provided['arch'] == 'densenet':
        model = models.densenet121()
    num_features = model.classifier[0].in_features
    from collections import OrderedDict
    for param in model.parameters():
        param.requires_grad = False
    classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(num_features, 512)),
                          ('relu', nn.ReLU()),
                          ('drpot', nn.Dropout(p=0.5)),
                          ('hidden', nn.Linear(512, hidden_units)),                        
                          ('fc2', nn.Linear(hidden_units, 102)),
                          ('output', nn.LogSoftmax(dim=1)),
                          ]))
    model.classifier = classifier
    model.load_state_dict(checkpoint_provided['state_dict'])
    if gpu:
        if use_gpu:
            model = model.cuda()
            print ("Using GPU")
        else:
            print("Using CPU")
    class_to_idx = checkpoint_provided['class_to_idx']
    idx_to_class = { v : k for k,v in class_to_idx.items()}
    return model, class_to_idx, idx_to_class

### Function To Show 5 Images with the Top Prediction Probabiliities 

In [6]:
def predict(image_path, model, class_to_idx, idx_to_class, cat_to_name, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    image = process_image(image_path)
    image.unsqueeze_(0)
    if use_gpu and gpu:
        model = model.cuda()    
    model.eval()
    if use_gpu and gpu:
        output = model.forward(image.cuda())
    else:
        output = model.forward(image)
    ps = torch.exp(output.cpu())#.data.numpy()[0]
    top_ps, top_labels = ps.topk(topk, dim=1)
    top_ps = top_ps.detach().numpy().tolist()[0]
    top_flowers = [cat_to_name[idx_to_class[x]] for x in top_labels.detach().numpy().tolist()[0]]
    return top_ps,top_flowers

### Declaring the Args to be Passed to the above Functions

In [13]:
gpu = True
hidden_units = 100
saved_model = "model.pth"
topk = 5

## Testing the Model

#### ![pink primrose](flowers/test/1/image_06743.jpg)

#### pink primrose

In [17]:
image_path = "flowers/test/1/image_06743.jpg" 

with open('cat_to_name.json', 'r') as f:
        cat_to_name = json.load(f)

model, class_to_idx, idx_to_class = load_checkpoint(saved_model)
top_probability, top_predictions = predict(image_path, model, class_to_idx, idx_to_class, cat_to_name, topk=topk)

print('Predicted Classes: ', top_predictions)
print('Predicted Probability: ', top_probability)
print ('predicted class {} with an accuracy of {}%'.format(top_predictions[0],top_probability[0]*100))

Using GPU
Predicted Classes:  ['pink primrose', 'hibiscus', 'watercress', 'mallow', 'tree mallow']
Predicted Probability:  [0.9598137140274048, 0.02773149497807026, 0.010170906782150269, 0.0016894926084205508, 0.0002148736675735563]
predicted class pink primrose with an accuracy of 95.98137140274048%


#### ![globe thistle](flowers/test/10/image_07090.jpg)

#### globe thistle

In [19]:
image_path = "flowers/test/10/image_07090.jpg" 

with open('cat_to_name.json', 'r') as f:
        cat_to_name = json.load(f)

model, class_to_idx, idx_to_class = load_checkpoint(saved_model)
top_probability, top_predictions = predict(image_path, model, class_to_idx, idx_to_class, cat_to_name, topk=topk)

print('Predicted Classes: ', top_predictions)
print('Predicted Probability: ', top_probability)
print ('predicted class {} with an accuracy of {}%'.format(top_predictions[0],top_probability[0]*100))

Using GPU
Predicted Classes:  ['globe thistle', 'artichoke', 'common dandelion', 'king protea', 'pincushion flower']
Predicted Probability:  [0.9998131394386292, 0.00017802120419219136, 7.51867582948762e-06, 1.2235684607730946e-06, 9.893702923591263e-08]
predicted class globe thistle with an accuracy of 99.98131394386292%


#### ![arum lily](flowers/test/20/image_04912.jpg)

#### arum lily

In [21]:
image_path = "flowers/test/20/image_04912.jpg" 

with open('cat_to_name.json', 'r') as f:
        cat_to_name = json.load(f)

model, class_to_idx, idx_to_class = load_checkpoint(saved_model)
top_probability, top_predictions = predict(image_path, model, class_to_idx, idx_to_class, cat_to_name, topk=topk)

print('Predicted Classes: ', top_predictions)
print('Predicted Probability: ', top_probability)
print ('predicted class {} with an accuracy of {}%'.format(top_predictions[0],top_probability[0]*100))

Using GPU
Predicted Classes:  ['giant white arum lily', 'moon orchid', 'frangipani', 'ruby-lipped cattleya', 'thorn apple']
Predicted Probability:  [0.9993442893028259, 0.00041622499702498317, 0.00014414452016353607, 3.6253604775993153e-05, 3.0203471396816894e-05]
predicted class giant white arum lily with an accuracy of 99.93442893028259%


#### ![sweet william](flowers/test/30/image_03466.jpg)

#### sweet william

In [22]:
image_path = "flowers/test/30/image_03466.jpg" 

with open('cat_to_name.json', 'r') as f:
        cat_to_name = json.load(f)

model, class_to_idx, idx_to_class = load_checkpoint(saved_model)
top_probability, top_predictions = predict(image_path, model, class_to_idx, idx_to_class, cat_to_name, topk=topk)

print('Predicted Classes: ', top_predictions)
print('Predicted Probability: ', top_probability)
print ('predicted class {} with an accuracy of {}%'.format(top_predictions[0],top_probability[0]*100))

Using GPU
Predicted Classes:  ['sweet william', 'barbeton daisy', 'tree mallow', 'carnation', 'petunia']
Predicted Probability:  [0.9739780426025391, 0.012665963731706142, 0.010556580498814583, 0.0008589239441789687, 0.000856724102050066]
predicted class sweet william with an accuracy of 97.3978042602539%


#### ![lenten rose](flowers/test/40/image_04563.jpg)

#### lenten rose

In [26]:
image_path = "flowers/test/40/image_04563.jpg" 

with open('cat_to_name.json', 'r') as f:
        cat_to_name = json.load(f)

model, class_to_idx, idx_to_class = load_checkpoint(saved_model)
top_probability, top_predictions = predict(image_path, model, class_to_idx, idx_to_class, cat_to_name, topk=topk)

print('Predicted Classes: ', top_predictions)
print('Predicted Probability: ', top_probability)
print ('predicted class {} with an accuracy of {}%'.format(top_predictions[0],top_probability[0]*100))

Using GPU
Predicted Classes:  ['lenten rose', 'columbine', 'hippeastrum', 'camellia', 'mallow']
Predicted Probability:  [0.9578976631164551, 0.03748911991715431, 0.004568890202790499, 3.2806296076159924e-05, 6.57527152725379e-06]
predicted class lenten rose with an accuracy of 95.78976631164551%
